In [6]:
# Ex1

from scipy.io import loadmat

mnist_path = "./mnist-original.mat"

mnist_raw = loadmat(mnist_path)
mnist = {
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0],
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
}
print("Done!")

Done!


In [7]:
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([0., 0., 0., ..., 9., 9., 9.])}

In [8]:
X, y = mnist['data'], mnist['target']
X_train, y_train, X_test, y_test = X[:60000], y[:60000], X[60000:], y[60000:]

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

neighbor_clf = KNeighborsClassifier(n_neighbors=6)
#neighbor_clf.fit(X_train, y_train)
#cross_val_score(neighbor_clf, X_train, y_train, cv=3, scoring="accuracy")

In [10]:
#neighbor_clf.predict([X_test[20], X_test[21], X_test[2203]])


In [11]:
print(y_test[20], y_test[21], y_test[2203])

0.0 0.0 2.0


In [12]:
#cross_val_score(neighbor_clf, X_train, y_train, cv=10, scoring="accuracy")

In [13]:
# regularize the training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [14]:
X_train_scaled

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
#cross_val_score(neighbor_clf, X_train_scaled, y_train, cv=3, scoring="accuracy")

In [16]:
# ex2 
from scipy.ndimage import interpolation

# direction is a pair of shift (x_axis, y_axis) for each axis
def shift_image_by_onepixel(image, direction):
    image_2d = image.reshape(28, 28)
    rotated_image_2d = interpolation.shift(image_2d, direction, cval=0)
    return rotated_image_2d.flatten()

def shift_images_by_onepixel(images, direction):
    images_2d = [image.reshape(28, 28) for image in images]
    rotated_images = [interpolation.shift(image_2d, direction, cval=0) for image_2d in images_2d]
    return [rotated_image.flatten() for rotated_image in rotated_images]

X_train, y_train, X_test, y_test = X[:60000], y[:60000], X[60000:], y[60000:]
orig_image = X_train[20]
#orig_image.reshape(28, 28)

In [17]:
rotate_up = shift_image_by_onepixel(orig_image, (-1, 0))
#rotate_up.reshape(28, 28)

In [18]:
rotate_down = shift_image_by_onepixel(orig_image, (0, 1))
#rotate_down.reshape(28, 28)

In [19]:
# Generate the extended dataset
X_train_up = shift_images_by_onepixel(X_train, (-1, 0))
X_train_down = shift_images_by_onepixel(X_train, (1, 0))
X_train_right = shift_images_by_onepixel(X_train, (0, 1))
X_train_left = shift_images_by_onepixel(X_train, (0, -1))

In [20]:
import numpy as np
X_train_extended = np.concatenate((X_train, X_train_up, X_train_down, X_train_right, X_train_left))

In [21]:
X_train_extended.shape

(300000, 784)

In [22]:
y_train_extended = np.concatenate((y_train, y_train, y_train, y_train, y_train))

In [23]:
y_train_extended.shape

(300000,)

In [24]:
neighbor_clf = KNeighborsClassifier(n_neighbors=5)
#neighbor_clf.fit(X_train_extended, y_train_extended)

In [25]:
#y_pred = neighbor_clf.predict(X_test)

In [26]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier()
sgd_clf.fit(X_train_extended, y_train_extended)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [27]:
y_pred = sgd_clf.predict(X_test)

In [29]:
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, y_pred, average="micro")

0.8632

In [31]:
sgd_clf.fit(X_train, y_train)
y_pred = sgd_clf.predict(X_test)
precision_score(y_test, y_pred, average="micro")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8522

In [ ]:
# Ex1. more: grid search to find the best hyperparameter for KNeighborClassifier
from sklearn.model_selection import GridSearchCV
classifier = KNeighborsClassifier()
parameters = { 'n_neighbors': [3, 5, 7, 9, 11], 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(classifier, parameters, cv=5)
#grid_search.fit(X_train_extended, y_train_extended)
#grid_search.best_params_